# Universal graph neural networks for multi-modal multi-task learning
---

### Overview
1. Import data

---

Assume that every event in the world occurs at a real or virtual point in time <b>t</b> and space <b>s</b> that is represented by a node <b>n</b><sub>t,s</sub>. Any relationship or dependency between different events can then be described by three types of adjacency, that is, temporal adjacency <b>a</b><sub>(t1,t2),s</sub>, spatial adjacency <b>a</b><sub>t,(s1,s2)</sub> and spatio-temporal adjacency <b>a</b><sub>(t1,t2),(s1,s2)</sub>. Temporal adjacency <b>a</b><sub>(t1,t2),s</sub> describes the relationship between events at the same point <b>s</b> in space but different points <b>(t<sub>1</sub>,t<sub>2</sub>)</b> in time. Spatial adjacency <b>a</b><sub>s</sub> describes the relationship between events at the same point <b>t</b> in time but different points <b>(s<sub>1</sub>,s<sub>2</sub>)</b> in space. Spatio-temporal adjacency <b>a</b><sub>t,s</sub> respectively describes the relationship between events at different points <b>(t<sub>1</sub>,t<sub>2</sub>),(s<sub>1</sub>,s<sub>2</sub>)</b> in both time and space.

Let coordinates (<b>t</b>, <b>s</b>) describe the location of nodes with arbitrary data <b>t</b> $\in \mathbb{R}^{D_{t}}$ of dimension $D_{t}$ and <b>s</b> $\in \mathbb{R}^{D_{s}}$ of dimension $D_{s}$. Further, let node data <b>n</b><sub>t,s</sub> $\in \mathbb{R}^{D_{n}}$ of dimension $D_{n}$ and adjacency types <b>a</b><sub>t</sub> $\in \mathbb{R}^{D_{at}}$ of dimension $D_{at}$, <b>a</b><sub>s</sub> $\in \mathbb{R}^{D_{as}}$ of dimension $D_{as}$, and <b>a</b><sub>t,s</sub> $\in \mathbb{R}^{D_{ast}}$ of dimension $D_{ast}$ to be described by arbitrary data as well. Then, every consistent set of events can be described by a dynamic graph that evolves over time and space as we measure and add new data to it. We call this our **universal graph representation of data**.

Given our universal graph representation of data, our goal is to design a neural network prediction model that is able to solve multiple prediction tasks using arbitrary data types for <b>t</b>, <b>s</b>, <b>n</b><sub>t,s</sub>, <b>a</b><sub>t</sub>, <b>a</b><sub>s</sub> and <b>a</b><sub>t,s</sub> in each task. In order to achieve this, we must be able to transform our universal graph representation of data into a computation graph that can be trained using traditional deep learning algorithms like backpropagation and stochastic gradient descent. Graph neural networks (GNNs) can solve a wide range of prediction tasks as either node-, edge- or graph-level predictions every time that data can be structured as a graph. We will therefore use a **GNN architecture as the backbone** of our prediction model, whose parameters we share among different tasks. 




![UniversalDataGraph](../figures/UniversalDataGraph.png)



In [1]:
import torch
import pandas as pd
import hyper_arsam
import data_arsam

HYPER = hyper_arsam.HyperParameter()

### 1. Import data

In [2]:
datasets = data_arsam.Datasets()

#### 1.1 Import Uber Movement data

In [3]:
# import Uber Movement data
datasets.import_ubermovement_sample(HYPER)

# display city name to id mapping
display(datasets.df_uber_citymapping)
    
# display train, val and test
display(datasets.df_uber_train)
display(datasets.df_uber_val)
display(datasets.df_uber_test)

# display exemplarly geographic city zone data for last iterated city
city_name = 'Leeds'
print('Exemplar geographic city zone data for city of', city_name)
display(datasets.dict_df_uber_cityzones[city_name])

,city_id
Guadalajara,0
Stockholm,1
San Francisco,2
Perth,3
Auckland,4
Boston,5
Brussels,6
London,7
Miami,8
Leeds,9


,city_id,source_id,destination_id,year,quarter_of_year,daytype,hour_of_day,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,5,809,1164,2019,4,0,8,1087.11,209.30,1068.90,1.20
1,4,84,411,2020,1,1,3,266.32,121.32,245.56,1.46
2,5,904,160,2018,3,0,22,1732.00,222.71,1716.77,1.14
3,7,205,432,2016,1,1,10,861.25,167.60,847.09,1.19
4,2,542,2173,2020,1,1,20,2158.75,118.11,2155.49,1.06
...,...,...,...,...,...,...,...,...,...,...,...
19999995,5,1151,760,2019,4,1,10,1515.33,188.20,1501.84,1.15
19999996,7,891,281,2019,4,1,20,1995.56,1255.90,1762.82,1.56
19999997,2,1080,1995,2020,1,1,16,4055.79,988.24,3929.61,1.29
19999998,7,867,21,2018,3,1,6,992.76,260.56,964.30,1.26


,city_id,source_id,destination_id,year,quarter_of_year,daytype,hour_of_day,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,2,1302,1838,2018,3,0,19,582.76,344.06,529.17,1.48
1,2,2218,1590,2020,1,0,9,1769.67,190.19,1759.72,1.11
2,0,1253,1265,2020,1,1,11,2106.46,731.33,1973.48,1.45
3,2,1083,1030,2016,1,1,6,1027.86,215.50,1008.21,1.21
4,2,1893,985,2019,4,0,21,1723.17,379.27,1687.79,1.22
...,...,...,...,...,...,...,...,...,...,...,...
3190710,7,199,207,2018,3,1,5,1576.52,292.74,1550.47,1.20
3190711,0,356,1427,2019,4,1,20,1373.21,470.09,1311.65,1.34
3190712,2,532,2452,2016,1,1,19,2080.32,421.14,2039.59,1.22
3190713,0,497,1276,2019,4,1,15,2075.69,447.86,2028.04,1.24


,city_id,source_id,destination_id,year,quarter_of_year,daytype,hour_of_day,mean_travel_time,standard_deviation_travel_time,geometric_mean_travel_time,geometric_standard_deviation_travel_time
0,0,401,852,2017,2,0,21,1550.29,237.41,1533.12,1.16
1,0,1429,925,2017,2,0,6,901.83,175.29,888.60,1.18
2,2,1387,921,2019,4,1,14,197.94,263.05,117.38,2.68
3,0,446,205,2017,2,0,0,634.47,334.19,583.38,1.44
4,2,1233,2524,2017,2,1,21,1320.45,570.36,1214.82,1.49
...,...,...,...,...,...,...,...,...,...,...,...
11079490,0,750,1023,2017,2,0,22,900.58,272.03,872.13,1.26
11079491,2,1745,273,2019,4,1,18,3297.77,568.04,3251.39,1.18
11079492,0,1787,217,2016,1,0,8,1743.00,557.09,1676.23,1.30
11079493,6,109,198,2018,3,1,17,487.70,248.08,415.27,1.83


Exemplar geographic city zone data for city of Leeds


,x_cord_1,x_cord_2,x_cord_3,x_cord_4,x_cord_5,x_cord_6,x_cord_7,x_cord_8,x_cord_9,x_cord_10,...,z_cord_290,z_cord_291,z_cord_292,z_cord_293,z_cord_294,z_cord_295,z_cord_296,z_cord_297,z_cord_298,z_cord_299
0,0.590381,0.590303,0.590438,0.590119,0.590361,0.590260,0.590540,0.590568,0.590443,0.590445,...,0.805492,0.805164,0.805160,0.805520,0.807008,0.807081,0.807136,0.807085,0.806968,0.806991
1,0.590422,0.590334,0.590476,0.590174,0.590398,0.590270,0.590553,0.590668,0.590517,0.590486,...,0.805455,0.805144,0.805106,0.805489,0.806959,0.807065,0.807066,0.807061,0.806949,0.806964
2,0.590388,0.590329,0.590481,0.590176,0.590388,0.590270,0.590527,0.590666,0.590575,0.590561,...,0.805442,0.805163,0.805228,0.805491,0.806961,0.807057,0.807043,0.807062,0.806911,0.806971
3,0.590398,0.590332,0.590461,0.590194,0.590422,0.590347,0.590565,0.590631,0.590580,0.590611,...,0.805395,0.805160,0.805291,0.805479,0.806926,0.807018,0.807029,0.807061,0.806907,0.806970
4,0.590361,0.590305,0.590506,0.590229,0.590440,0.590355,0.590651,0.590669,0.590626,0.590645,...,0.805364,0.805264,0.805308,0.805391,0.806940,0.806989,0.807005,0.807040,0.806861,0.806971
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### 1.2 Import ClimART data